<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_20%ED%8C%8C%EC%9D%B4%EC%8D%AC(%EC%97%B0%EA%B4%80%EC%84%B1%EB%B6%84%EC%84%9D_apriori_algorithm%2C_%EC%98%81%EC%96%B4%ED%92%88%EC%82%AC%ED%83%9C%EA%B9%85_nltk).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
from pandas import DataFrame, Series

teens = pd.read_csv('c:/data/snsdata.csv')
teens.info()

# 결측값
teens.isnull().sum()

teens['gender'].value_counts()
teens['gender'].value_counts(dropna=True)
teens['gender'].value_counts(dropna=False) # 결측값까지 춫력
teens['gender'].isnull().sum()
teens['gender'].describe()
teens['gender'].value_counts(dropna=False).plot(kind='bar') # 결측값까지 춫력

teens_df = pd.DataFrame(teens['gender'].value_counts(dropna=False)).reset_index()
teens_df.columns = ['gender','freq']
teens_df
teens_df.loc[teens_df.gender.isnull(),'gender'] = 'NA'

import seaborn as sns
sns.barplot(x='gender',y='freq', data=teens_df)

teens['age'].isnull().sum()
teens['age'].describe()  # 숫자데이터들을 활용할 때 describe함수로 확인해보기
teens.describe()

teens.boxplot(column='age')
sns.boxplot(x=teens['age'])
sns.boxplot(y=teens['age'])


len([i for i in teens['age'] if (i < 13) | (i >= 20)])
teens['age']=[np.nan if (i < 13) | (i >= 20) else i for i in teens['age']]
teens['age'].describe()
teens['age'].isnull().sum()

# 졸업년도를 기준으로 평균나이
teens[['gradyear','age']]
gradage = teens['age'].groupby(teens['gradyear']).mean()
gradage.index

teens['agemean'] = [gradage.loc[i] for i in teens['gradyear']]
teens

teens['age'] = np.where(teens['age'].isnull(),teens['agemean'],teens['age']) # np.where(조건,조건이true일때 수행, false일때 수행)
teens['age'].isnull().sum()
teens['age'].describe()

teens['female'] = [1 if i == 'F' else 0 for i in teens['gender']] # 수치화
teens['female'].value_counts()

teens['no_gender'] = [1 if pd.isnull(i) else 0 for i in teens['gender']]
teens['no_gender'].value_counts()

teens.info()
data = teens.iloc[:,4:40]

from sklearn.cluster import KMeans
model = KMeans(n_clusters=5)
model.fit(data)
model.labels_
model.cluster_centers_
model.cluster_centers_.shape
teens['cluster'] = model.labels_
teens.loc[1:5,['cluster','gender','age','friends']]

teens['age'].groupby(teens['cluster']).mean()
teens['female'].groupby(teens['cluster']).mean()
teens['friends'].groupby(teens['cluster']).mean()

for name,group in teens.groupby('cluster'):
    print(name)
    print(group)
    
teens[teens['cluster'] == 0][teens.columns]
col = teens.columns.difference(['gradyear','gender','age','agemean','female', 'no_gender', 'cluster','friends'])
teens_0 = teens[teens['cluster'] == 0][col] # columns.difference([제외할 컬럼]) : 제외할 컬럼 빼고 추출
teens_0.sum().sort_values(ascending=False)[:10]

teens_0 = teens[teens['cluster'] == 1][col]
teens_0.sum().sort_values(ascending=False)[:10]

teens_0 = teens[teens['cluster'] == 2][col]
teens_0.sum().sort_values(ascending=False)[:10]

teens_0 = teens[teens['cluster'] == 3][col]
teens_0.sum().sort_values(ascending=False)[:10]

teens_0 = teens[teens['cluster'] == 4][col]
teens_0.sum().sort_values(ascending=False)[:10]

data['cluster'] = model.labels_
data.columns[:-1] # 마지막 열은 제외

data.columns[:-1][model.cluster_centers_.argsort()[:,::-1][0][:10]] # argsort() 인덱스 값을 추출
data.columns[:-1][model.cluster_centers_.argsort()[:,::-1][1][:10]]
data.columns[:-1][model.cluster_centers_.argsort()[:,::-1][2][:10]]
data.columns[:-1][model.cluster_centers_.argsort()[:,::-1][3][:10]]
data.columns[:-1][model.cluster_centers_.argsort()[:,::-1][4][:10]]

ks = range(1,10)
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(data)
    inertias.append(model.inertia_)

# Plot ks vs inertias
plt.plot(ks, inertias,'-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.ticklabel_format(style='plain')
plt.show()


★ 연관성분석(association analysis)
- 대량의 데이터에 숨겨진 항목간의 연관규칙을 찾아내는 기법으로서 장바구니분석(market basket analysis)이라고도 함
- 실제 연관성분석은 모든 기업에서 다양한 마케팅활동에 활용하고 있으며 더 나아가 사회 네트워크 분석에도 활용할 수 있다.

장점
- 대규모 거래 데이터에 대해 작업할 수 있다.
- 이해하기 쉬운 규칙을 생성한다
- 데이터마이닝과 데이터베이스에서 예상치 못한 지식을 발굴하는데 유용

단점
- 작은데이터셋에는 그다지 유용하지 않다.
- 진정한 통찰력과 상식을 분리하기 위한 노력이 필요하다.

지지도(support)
- 전체 거래중 연관성 규칙을 구성하는 항목들이 포함된 거래의 비율
- 전체 거래중 연관성 규칙이 얼마나 빈번히 발생하는지 확인하는 지표

            항목에 대한 거래 갯수
support = ----------------------
            전체 거래수
            
신뢰도(confidence)
- 조건이 발생했을 때 동시에 일어날 확률을 의미하며 신뢰도가 1에 가까울수록 의미있는 연관성을 가지고 있다.
- 조건을 포함하는 거래중 연관성 규칙이 얼마나 빈번히 발생하는지 확인하는 지표

{조건} -> {결과}

                조건과 결과 항목을 포함하는 거래수
confidence = --------------------------
                조건항목을 포함한 거래수

                            support(x,y)
confidence(x -> y) = -------------------------
                            support(x)


거래번호    구매물품
--------    --------------------------
1           - 우유, 버터, 시리얼
2           - 우유, 시리얼
3           - 우유, 빵
4           - 버터, 맥주, 오징어

support(우유 -> 시리얼) : 우유와 시리얼을 동시에 구매할 확률, {우유,시리얼}포함한 거래수 / 전체거래수 ,2/4
confidence(우유 -> 시리얼) : 우유를 구매할 때 시리얼 같이 구매할 조건부확률, {우유,시리얼} / {우유}

support(우유 -> 시리얼)  = 50%
confidence(우유 -> 시리얼) = 66.7%

support(시리얼 -> 우유) = 50%
confidence(시리얼 -> 우유) = 100%

향상도(lift)
- 지지도와 신뢰도를 동시에 고려하는 지표

                    confidence(시리얼 -> 우유)
lift(시리얼,우유) = -------------------------------
                            support(우유)
                            
또는
                    support(시리얼 -> 우유)         
lift(시리얼,우유) = ------------------------------- 
                    support(시리얼) * support(우유)
                            
                    confidence(시리얼 -> 우유)              1
lift(시리얼,우유) = ------------------------------- =  ---------- = 1/0.75 = 1.33
                    support(우유)                          3/4

향상도값이 1인 경우 조건과 결과는 우연에 의한 관계라고 보여,1보다 클수록 우연이 아닌 의미있는 연관성을 가진 규칙으로 해석하면 된다.

거래번호    구매물품
--------    --------------------------
1           A, C, D
2           B, C, E
3           A, B, C, E
4           B, E

항목      지지도(항목구매건수/전체구매건수)
------- -------------
A       2
B       3
C       3
D       1
E       3

지지도 2이상인 항목만 추출

항목      지지도(항목구매건수/전체구매건수)
------- -------------
A       2
B       3
C       3
E       3

항목      지지도
------- --------------
A B     1    
A C     2
A E     1
B C     2
B E     3
C E     2

지지도 2 이상인 항목 추출

항목      지지도
------- --------------
A C     2
B C     2
B E     3
C E     2

각각 항목에서 첫번째 항목을 기준으로 동일한 것을 찾아보세요.
B C E

거래번호    구매물품
--------    -------------------------- 
1           A, C, D
2           B, C, E
3           A, B, C, E
4           B, E

발견된 규칙항목        지지도
----------------    ---------
B, C, E                 2

★ apriori algorithm
- 집합의 크기가 1인경우부터 차례로 늘려가면서 처리하는 알고리즘
- k 개인 빈도가 높은 항목을 구했다면 그 다음에는 k+1인 항목의 집합을 계산한다.


dataset = [['우유','버터','시리얼'],
           ['우유','시리얼'],
           ['우유','빵'],
           ['버터','맥주','오징어']]

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

te = TransactionEncoder()
te_data = te.fit_transform(dataset)
df = pd.DataFrame(te_data,columns = te.columns_)
f = apriori(df,min_support = 0.1, use_colnames=True)
association_rules(f)
association_rules(f,metric='confidence')

import csv
groceries = []
with open('c:/data/groceries.csv','r') as file:
    csv_data = csv.reader(file)
    for row in csv_data:
        groceries.append(row)
        
te = TransactionEncoder()
te_data = te.fit_transform(groceries)
df = pd.DataFrame(te_data,columns = te.columns_)
f = apriori(df,min_support = 0.005, use_colnames=True)
#association_rules(f)  - min_threshold값을 조정해야 값이 나옴( min_threshold값이 높으면 값들이 나올 수 없기 때문)
rules = association_rules(f,metric='confidence', min_threshold=0.01)
rules[rules['antecedents'] == {'whole milk','butter'}]


import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

mom = pd.read_csv('c:/data/mom.txt',header=None,names=['sentens'])
mom

txt = ' '.join(mom['sentens'])
txt
word_tokenize(txt)
words = word_tokenize(txt)
pos_tag(txt) # 문장은 품사 태깅할 때 오류!
pos_tag(words) # 단어 태깅 가능!( word_tokenize로 단어로 바꿔주고 품사 태깅하기 )
# nltk에서 품사태깅하려면 단어로 분리한 후 수행해야 한다.
pos_tag(word_tokenize(txt))

# 품사정보를 확인
nltk.download('tagsets')
nltk.help.upenn_tagset('NNS')

tagg_list = pos_tag(word_tokenize(txt))
tagg_list

nouns_list = [i[0] for i in tagg_list if i[1] == 'NN' ]
nouns_list

nouns_list = [i[0] for i in tagg_list if i[1] in ['NN','NNP'] ]
nouns_list

from nltk.tag import untag
untag(tagg_list) # 태그 정보 빼고 출력

